**<div align="center"><span style="font-size:4em">Lösung</span></div>**
# Programmieraufgabe: 3D-Printer

Eine kleine Firma betreibt professionellen 3D-Druck. In der Woche kann die Firma den 3D-Drucker $T$ Stunden betreiben. Die Aufträge $A_1,\ldots, A_n$ werden der Firma angeboten. Die Firma will gewisse dieser Aufträge auswählen und die anderen ablehnen (oder auf später verschieben), und so den Wochengewinn maximieren. Jeder Auftrag $A_i$ ist durch eine Bearbeitungsdauer $d_i$ (in Stunden) und einen Gewinn $p_i$ (in Euro) beschrieben. Dazu kommen gewisse technische Unterschiede in den Aufträgen, die gegebenfalls eine Reinigung oder einen Umbau des 3D-Druckers bedingen. Wir werden diese Unterschiede der Einfachheit unter dem Stichwort Substrat $s_i$ zusammenfassen: Ein Auftrag braucht vielleicht nur schnödes graues Plastik, ein anderer eine Plastik-Metallmischung in orange. (Ich habe keine Ahnung wie 3D-Drucker funktionieren.)


### Aufgabe (a): Einfaches MIP
Implementieren Sie ein MIP und lösen Sie <code>simple_instance</code>. Führen Sie dazu Indikatorvariablen <code>x[i]</code> ein, die angeben, ob der Auftrag <code>i</code> angenommen wird oder nicht. Führen Sie zusätzlich eine Variable <code>profit</code> ein, die den Gewinn beziffert. Nutzen Sie den Code unten, um die angenommenen Aufträge und den Gewinn auszugeben.

In [1]:
def simple_instance():
    T = 30
    d = [2,3,6,3,4,7,4,2,4,5]
    p = [22,31,65,33,42,71,41,22,30,59]
    return T,d,p

Wie immer muss das Paket <code>mip</code> installiert sein. In Colab (oder auf Ihrem eigenen Rechner) installieren Sie <code>mip</code> gegebenfalls durch Löschen der Raute unten. 

In [2]:
# Raute Löschen, um mip zu installieren
# !pip install mip
import mip

### Lösung (a)

Als erstes instantiieren wir das Modell und führen Indikatorvariablen $x_i$ ein, die angeben, ob wir Auftrag $A_i$ annehmen oder nicht. Wir definieren zudem eine Variable für den Gewinn.

In [3]:
T,d,p=simple_instance()
n=len(d)

model=mip.Model()
# Indikator x[i], ob Auftrag i ausgeführt wird
x = [model.add_var(var_type=mip.BINARY) for _ in range(n)] 
profit = model.add_var() # automatically non-negative

Nun die einzige Bedingung: Wir dürfen die Produktionszeit des 3D-Druckers nicht übersteigen. Als zusätzliche Bedingung berechnen wir den Gewinn.

In [4]:
model+= mip.xsum( d[i]*x[i] for i in range(n)) <= T
profit = mip.xsum(p[i]*x[i] for i in range(n))

Zielfunktion ist nun einfach die Maximierung von <code>profit</code>. Wir führen die Optimierung dann auch gleich durch.

In [5]:
model.objective=mip.maximize(profit)
model.verbose=0 # suppress lengthy solver output
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

Schließlich die Ausgabe der Lösung.

In [6]:
accepted_jobs=[i for i in range(n) if x[i].x>0]
print("Angenommene Aufträge: {}".format(accepted_jobs))
print("Gewinn: {}EUR".format(profit.x))

Angenommene Aufträge: [0, 1, 2, 3, 4, 5, 9]
Gewinn: 323.0EUR


## MIP mit Rüstzeiten
Das Modell bisher war sehr einfach. In der Realität kommen oft Rüstzeiten dazu: Der 3D-Drucker muss umgerüstet, eventuell gereinigt werden, wenn sich das Substrat ändert. Bestand etwa bei einem Auftrag das Substrat aus grauem Plastik und soll nun der folgende Auftrag mit weißem Plastik durchgeführt werden, so muss der Drucker gereinigt werden, um Farbfehler zu vermeiden. Wie viel Zeit auf die Umrüstung / Reinigung verwendet werden muss, hängt dabei davon ab, von welchem Substrat auf welches Substrat gewechselt wird. Konkret: Im Gegensatz zu vorher muss nun eine Reihenfolge der angenommenen Aufträge bestimmt werden, und wenn dann auf einen Auftrag $A_i$ mit Substrat $k=s_i$ der Auftrag $A_j$ mit $\ell=s_j$ folgt, wird eine Rüstzeit von $r_{k,\ell}$ nötig.

Es sei $\{1,\ldots, C\}$ die Menge der unterschiedlichen Substrate. Für die Rüstzeiten soll gelten:
* $r_{k,\ell}\geq 0$ für alle $k,\ell\in \{1,\ldots, C\}$, dh, keine Reisen in die Vergangenheit.
* $r_{k,k}=0$ für all $k\in\{1,\ldots, C\}$, dh wenn das Substrat nicht gewechselt wird, fallen keine Rüstzeiten an.
* es gilt eine Dreiecksungleichung $r_{k,m}\leq r_{k,\ell}+r_{\ell,m}$ für alle $k,\ell,m\in\{1,\ldots, C\}$, dh, Rüstzeiten können nicht verkürzt werden, in dem auf ein drittes Substrat gewechselt wird. (Ist ja klar.)

Es kann aber durchaus $r_{k,\ell}\neq r_{\ell,k}$ gelten, dh, die Rüstzeiten sind nicht notwendig symmetrisch. Der Wechsel von weißem Substrat auf graues erfordert eventuell eine kürzere Reinigung als umgekehrt.

Am Ende ist keine Reinigung nötig. (Ich weiß, igitt.)

### Aufgabe (b): Reihenfolge 
Argumentieren Sie, dass alle (angenommenen) Aufträge mit gleichem Substrat konsekutiv ausgeführt werden können. Dh, es reicht aus, die Substrate der angenommenen Aufträge in eine Reihenfolge zu bringen. Schreiben Sie Ihre Argumentation in die folgende *markdown*-Zelle.

### Lösung (b)
Sei $R$ eine Reihung der angenommenen Aufträge, in der die Aufträge mit gleichem Substrat nicht konsekutiv sind. 
Dann gibt es in $R$ Aufträge $A_{i_1},A_{i_2},A_{i_3}$ mit Substraten $\ell=s_{i_1}=s_{i_3}$ und $k=s_{i_2}\neq \ell$, so dass die zeitliche Reihung wie folgt ist:
$$R:\:\ldots,A_{i_1},\ldots,A_{i_2}, A_{i_3},A_{i_4}\ldots$$
Dabei deuten Punkte mögliche weitere Aufträge an und $A_{i_4}$ sei der auf $A_{i_3}$ folgende Auftrag. Wenn $A_{i_3}$ der letzte Auftrag ist, so fügen wir einen künstlichen Auftrag $A_{i_4}$ hinzu, der eine Beabeitungsdauer von $0$ hat und keinerlei Umrüstung erfordert (Rüstzeit $0$, egal welcher Auftrag davor kommt).

Wir sortieren um: der Auftrag $A_{i_3}$ soll nun direkt nach $A_{i_1}$ ausgeführt werden:
$$R':\:\ldots,A_{i_1},A_{i_3}\ldots,A_{i_2},A_{i_4},\ldots$$
Wie ändern sich die Rüstzeiten? Da $r_{\ell,\ell}=0$, entstehen keine neuen Rüstzeiten, wenn $A_{i_3}$ direkt auf $A_{i_1}$ folgt. Bei $A_{i_2}$ ändert sich jedoch etwas: Sei $m=s_{i_4}$ das Substrat von $A_{i_4}$. Dann fielen beim Wechsel $A_{i_2}\rightarrow A_{i_3}\rightarrow A_{i_4}$ in $R$ die  Rüstzeiten $r_{k,\ell}+r_{\ell,m}$ an. In $R'$ hingegen fallen nur Rüstzeiten bei $A_{i_2}\rightarrow A_{i_4}$ von $r_{k,m}$ an. Da, nach Dreiecksungleichung, $r_{k,m}\leq r_{k,\ell}+r_{\ell,m}$ erhöhen sich die Rüstzeiten insgesamt nicht. So können wir also schrittweise die Aufträge umsortieren bis identische Substraten konsekutiv in den Aufträgen auftreten. 

### Aufgabe (c): MIP mit Rüstzeiten
Implementieren Sie das MIP mit Rüstzeiten und lösen Sie <code>small_instance</code> oder <code>large_instance</code>. 

Anleitung: Wie oben führen Sie Variablen <code>x[i]</code> und <code>profit</code> ein. Führen Sie zusätzlich für $k,p=1,\ldots,C$ eine Indikatorvariable <code>z[k][p]</code> ein, die bestimmt, ob das Substrat $k$ an $p$ter Stelle der Reihung der Substrate stehen soll. Die Variable <code>R</code> soll die Gesamtrüstzeit bezeichnen. Nutzen Sie den Code unten, um die angenommenen Aufträge, die Reihung der Substrate, die Gesamtrüstzeit und den Gewinn auszugeben. Sie werden sicher noch weitere Variablen einführen müssen.

In [7]:
def small_instance():
    T,d,p=simple_instance()
    s = [1, 2, 0, 2, 3, 0, 3, 1, 3, 4]
    r = [[0, 6, 8, 7, 7],
         [6, 0, 5, 4, 4],
         [5, 5, 0, 4, 4],
         [4, 4, 6, 0, 7],
         [8, 6, 7, 7, 0]]
    C=max(s)+1
    return T,d,p,s,r,C

def large_instance():
    T = 60
    d = [2,3,6,3,4,7,4,2,4,5,6,2,5,7,2,3,4,3,7,8,6,6,2,3,5,9,6,7,4,1]
    p = [22,31,65,33,42,71,41,22,30,59,63,22,51,72,27,38,46,36,77,84,64,64,23,32,53,91,61,71,43,15]
    s = [1, 2, 0, 2, 3, 0, 3, 1, 3, 4, 1, 1, 4, 0, 1, 2, 3, 2, 0, 0, 4, 4, 3, 2, 1, 0, 1, 2, 3, 0]
    r = [[0, 6, 8, 7, 7],
         [6, 0, 5, 4, 4],
         [5, 5, 0, 4, 4],
         [4, 4, 6, 0, 7],
         [8, 6, 7, 7, 0]]
    C=max(s)+1
    return T,d,p,s,r,C

### Lösung (c)
Wir stellen zunächst das MIP auf:
\begin{align}
\max\quad & \sum_{i=1}^np_ix_i & \\
\textsf{unter}\quad & \sum_{i=1}^n d_ix_i+R\leq T & (1) \\
& R=\sum_{k=1}^C\sum_{\ell=1}^Cr_{k,\ell}z_{k,\ell} & (2) \\
& z_{k,\ell}\geq y_{k,p}+y_{\ell,p+1}-1\quad\textsf{ für alle }p=1,\ldots, C-1\textsf{ und }k,\ell=1,\ldots, C & (3) \\
& x_i\leq \sum_{p=1}^Cz_{s_i,p}\quad\textsf{ für alle }i=1\ldots,n & (4) \\
& z_{k,p+1}\leq z_{k,p}\quad\textsf{ für alle }p=1,\ldots, C-1\textsf{ und }k=1,\ldots, C & (5)\\
& x_i,y_{k,\ell},z_{k,p}\in\{0,1\}\quad\textsf{ für alle }i=1,\ldots n\textsf{ und }k,\ell,p=1,\ldots,C & (6)
\end{align}

Was machen die einzelnen Variablen und Bedingungen? Wie oben ist
$$
x_i=\begin{cases} 
1 & \textsf{ Auftrag $A_i$ wird angenommen}\\
0 & \textsf{ Auftrag $A_i$ wird abgelehnt}
\end{cases}
$$
Damit drückt die Zielfunktion wieder den Gewinn aus. Wir werden $R$ als die Gesamtrüstzeit identifizieren. Damit ist (1) die Kapazitätsbeschränkung: Gesamtbearbeitungszeit plus Gesamtrüstzeit darf die verfügbare Zeit $T$ nicht überschreiten. 

Wir werden erzwingen, dass 
$$
z_{k,\ell}=\begin{cases} 
1 & \textsf{ Substrat $\ell$ folgt direkt auf Substrat $k$}\\
0 & \textsf{ sonst}
\end{cases}
$$
Damit ist mit (2) $R$ einfach die Gesamtrüstzeit. Um das Versprechen bezüglich $z_{k,\ell}$ einzulösen, fügen wir die Variablen $y_{k,p}$ ein. Diese sollen folgendes erfüllen:
$$
y_{k,p}=\begin{cases} 
1 & \textsf{ Substrat $k$ ist an Position $p$ der Reihung}\\
0 & \textsf{ sonst}
\end{cases}
$$
Ist das der Fall, so sagt (3) aus, dass der Substratwechsel $k\rightarrow \ell$ auftritt, wenn es eine Position $p$ gibt, so dass $k$ an Position $p$ auftritt und $\ell$ an Position $p+1$. Nun müssen wir noch die Substrate mit der Auftragsannahme verknüpfen. Dies geschieht in (4): Wenn der Auftrag $A_i$ angenommen wird, dann muss das entsprechende Substrat an einer Position der Reihung auftreten. Schließlich darf es keine Lücken in der Reihung geben, dh, es darf nicht vorkommen, dass an Position $p$ ein Substrat gereiht ist, an Position $p+1$ aber keines -- dies wird mit (5) erzwungen.

Implementieren wir das MIP! Dazu kopieren wir zunächst die Teile aus (a), die wir wiederverwenden können.

In [8]:
T,d,p,s,r,C=small_instance()
#T,d,p,s,r,C=large_instance()
n=len(d)

model=mip.Model()
# Indikator x[i], ob Auftrag i ausgeführt wird
x = [model.add_var(var_type=mip.BINARY) for _ in range(n)] 
profit = model.add_var() # automatically non-negative
profit = mip.xsum(p[i]*x[i] for i in range(n))
model.objective=mip.maximize(profit)

Als nächstes fügen wir die Variablen $R$, $y_{k,\ell}$ und $z_{k,p}$ hinzu.

In [9]:
y=[[model.add_var(var_type=mip.BINARY) for _ in range(C)] for _ in range(C)]
z=[[model.add_var(var_type=mip.BINARY) for _ in range(C)] for _ in range(C)]
R=model.add_var()

Bedingung (1): Es steht nur die Zeit $T$ zur Verfügung.

In [10]:
model += mip.xsum( d[i]*x[i] for i in range(n))+ R <= T

Bedingung (2): $R$ ist die Gesamtrüstzeit.

In [11]:
model += R==mip.xsum( r[k][l]*z[k][l] for k in range(C) for l in range(C) )

Bedingung (3): Der Wechsel $k\rightarrow \ell$ tritt auf, wenn es ein $p$ gibt, so dass $k$ an Stelle $p$ und $\ell$ an Stelle $p+1$ gereiht ist.

In [12]:
for p in range(C-1):
    for k in range(C):
        for l in range(C):
            model += y[k][p]+y[l][p+1] <= z[k][l]

Bedingung (4): Wenn der Auftrag $i$ das Substrat $k$ benötigt, dann muss es gereiht werden.

In [13]:
for i in range(n):
    k=s[i]
    model += mip.xsum( z[k][p] for p in range(C)) >= x[i]

Bedingung (5): Die Reihung darf keine Lücken enthalten.

In [14]:
for p in range(C-1):
    for k in range(C):
        model += z[k][p+1] <= z[k][p]

Wir starten die Optimierung:

In [15]:
model.verbose=0 # suppress lengthy solver output
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

Ausgabe der Lösung

In [16]:
sequence=[k for p in range(C) for k in range(C) if z[k][p].x>0] 
jobs=[i for i in range(n) if x[i].x>0]
setup_time=R.x
profit_opt=profit.x
print("Angenommene Aufträge: {}".format(jobs))
print("Reihenfolge der Substrate: {}".format(sequence))
print("Gesamtrüstzeit: {}".format(setup_time))
print("Gewinn: {}EUR".format(profit_opt))

Angenommene Aufträge: [2, 4, 5, 6, 8]
Reihenfolge der Substrate: [0, 3]
Gesamtrüstzeit: 4.0
Gewinn: 249.0EUR
